In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

from epsampling.utils import load_csv
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
from epsampling.utils import drop_sers_with_nans
from epsampling.utils import date_str_to_int

DATA_DIR = '/work/users/k/4/k4thryn/Repos/EpSampling/data/'
DT = datetime.today().strftime('%Y%m%d-%H%M%S')

In [2]:
timestamp = '20241009-144131'

fpath = os.path.join(DATA_DIR,'processed', f'processed_naive_deaths_{timestamp}.csv')
df_deaths = pd.read_csv(fpath)
# display(df)

## Drop rows with nans.
df_deaths.dropna(inplace=True)
# display(df)

## Drop samples with negative inc deaths.
df_deaths = df_deaths[df_deaths.True_inc_deaths >= 0]
# display(df_deaths)

# # # # # # # # # # # # # # # # # # # #
# # # # # # # # # # # # # # # # # # # # 

from epsampling.utils import drop_duplicate_cols

# timestamp = '20241009-143022' 

fpath = os.path.join(DATA_DIR,'processed',f'formatted_acs_results_normed_{timestamp}.csv')
df_acs = pd.read_csv(fpath,index_col='Fips')

# Remove duplicate columns
df_acs = drop_duplicate_cols(df_acs)

In [3]:
df = df_deaths.merge(df_acs, on='Fips')
display(df)

,Fips,State_fips,Pop,State_pop,Ratio,Date,Proj_inc_deaths,True_cum_deaths,Cum_deaths_tm1,True_inc_deaths,...,HU_OCC,POP_NH,HINS_A0018,HINS_A65p,HH_x5,POP_A3564,HU_x14,POP_16p_EMP_x7,POP_A25p,POP_x2
0,1001,1,58239.0,4997675,0.011653,20200620,83.302156,9.0,6.0,3.0,...,0.904261,0.969522,0.979290,0.998043,21856.0,22690.0,24170.0,25871.0,39614.0,58239.0
1,1001,1,58239.0,4997675,0.011653,20200627,68.179357,12.0,9.0,3.0,...,0.904261,0.969522,0.979290,0.998043,21856.0,22690.0,24170.0,25871.0,39614.0,58239.0
2,1001,1,58239.0,4997675,0.011653,20200704,76.238975,13.0,12.0,1.0,...,0.904261,0.969522,0.979290,0.998043,21856.0,22690.0,24170.0,25871.0,39614.0,58239.0
3,1001,1,58239.0,4997675,0.011653,20200711,88.362530,15.0,13.0,2.0,...,0.904261,0.969522,0.979290,0.998043,21856.0,22690.0,24170.0,25871.0,39614.0,58239.0
4,1001,1,58239.0,4997675,0.011653,20200718,114.279381,21.0,15.0,6.0,...,0.904261,0.969522,0.979290,0.998043,21856.0,22690.0,24170.0,25871.0,39614.0,58239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304412,56045,56,6891.0,576641,0.011950,20220409,8.000000,18.0,18.0,0.0,...,0.802681,0.981425,0.844977,1.000000,2754.0,2724.0,3431.0,3044.0,4892.0,6891.0
304413,56045,56,6891.0,576641,0.011950,20220416,5.000000,18.0,18.0,0.0,...,0.802681,0.981425,0.844977,1.000000,2754.0,2724.0,3431.0,3044.0,4892.0,6891.0
304414,56045,56,6891.0,576641,0.011950,20220423,4.000000,18.0,18.0,0.0,...,0.802681,0.981425,0.844977,1.000000,2754.0,2724.0,3431.0,3044.0,4892.0,6891.0
304415,56045,56,6891.0,576641,0.011950,20220430,4.000000,18.0,18.0,0.0,...,0.802681,0.981425,0.844977,1.000000,2754.0,2724.0,3431.0,3044.0,4892.0,6891.0


In [4]:
df['Target'] = df.apply(lambda x: x.True_inc_deaths / x.Pop * 100000, axis=1)
df['Naive'] = df.apply(lambda x: x.Naive_inc_deaths / x.Pop * 100000, axis=1)
df['State'] = df.apply(lambda x: x.Proj_inc_deaths / x.State_pop * 100000, axis=1)
df['Cumtm1'] = df.apply(lambda x: x.Cum_deaths_tm1 / x.Pop * 100000, axis=1)

# df['Target'] = df.apply(lambda x: x.True_inc_deaths, axis=1)
# df['Naive'] = df.apply(lambda x: x.Naive_inc_deaths, axis=1)
# df['State'] = df.apply(lambda x: x.Proj_inc_deaths, axis=1)
# df['Cumtm1'] = df.apply(lambda x: x.Cum_deaths_tm1, axis=1)

In [5]:
# # min-max normalization
# df = (df - df.min()) / (df.max() - df.min())

In [6]:
def get_chunks(lst, n):
    """Return successive n-sized chunks from lst."""
    chunks = []
    for i in range(0, len(lst), n):
        chunks.append(lst[i:i + n])
    return chunks

In [7]:
chunks = get_chunks(list(df.Date.unique()), 4)

# idc = [2,10,18]
# idc = [10]

# for idx in idc:
    
idx = 18 #10

dates_test = chunks[idx]
dates_train = [x for x in df.Date.unique() if x not in dates_test]

df_train = df[df.Date.isin(dates_train)]
df_test = df[df.Date.isin(dates_test)]

feats = list(df_acs.columns) + ['State','Cumtm1','Naive',
                               'State_pop','Ratio','Pop']
print(feats)

X_train = df_train[feats]
X_test = df_test[feats]

y_train = df_train['Target']
y_test = df_test['Target']

y_naive = df_test['Naive']

['POP_NHPINH', 'POP_NHPI', 'POP_AIANNH', 'POP_AIAN', 'IND_AFFHM_MQE', 'HU_UIS02', 'HU_OCC_OPRgt200', 'IND_PSMAW_M', 'OCC_MBSA_ELCAM_L', 'OCC_SERV_PS_LE', 'HU_OCC_OPR151200', 'OCC_NRCM_FFF_x11', 'IND_AFFHM_AFFH', 'IND_TWU_U', 'IND_AFFHM', 'HU_OCC_OPR101150', 'OCC_MBSA_CES_LPSS', 'HH_7pP', 'OCC_MBSA_ELCAM_ADESM', 'POP_ASIANNH', 'POP_ASIAN', 'EDU_DOCT', 'HU_UIS2049', 'IND_INF_x7', 'HU_UIS50P', 'POP_A21', 'POP_A20', 'HU_UIS01A', 'EDU_PROF', 'POP_A8084', 'POP_A85p', 'HU_UIS1019', 'HU_UIS0304', 'POP_A6566', 'OCC_MBSA_CES_AE', 'OCC_MBSA_CES_CM', 'OCC_SERV_PS_FP', 'IND_FIRR_RR', 'POP_A1819', 'OCC_SERV_PS', 'OCC_SERV_HCS_x11', 'HH_6P', 'HU_UIS0509', 'OCC_SERV_BGM_x11', 'POP_A6061', 'OCC_MBSA_ELCAM_CSS', 'OCC_MBSA_HCPT_HTT', 'POP_A6769', 'IND_AERAF_AER', 'POP_OTH2PLNH', 'IND_WHT_x7', 'POP_HISP_x12', 'HHI_35_40k', 'OCC_SERV_PCS_x11', 'OCC_PTMM_MM_x11', 'POP_A7579', 'POP_A6264', 'HHI_30_35k', 'HHI_45_50k', 'IND_TWU_TW', 'OCC_MBSA_HCPT_HDTP', 'IND_PSMAW_ASWM', 'POP_A2224', 'POP_OTH2PL', 'OCC_NRCM_I

In [8]:
X_train

,POP_NHPINH,POP_NHPI,POP_AIANNH,POP_AIAN,IND_AFFHM_MQE,HU_UIS02,HU_OCC_OPRgt200,IND_PSMAW_M,OCC_MBSA_ELCAM_L,OCC_SERV_PS_LE,...,HU_x14,POP_16p_EMP_x7,POP_A25p,POP_x2,State,Cumtm1,Naive,State_pop,Ratio,Pop
0,0.000000,0.000000,0.001683,0.001683,0.001933,0.002193,0.002317,0.002358,0.002706,0.00286,...,24170.0,25871.0,39614.0,58239.0,1.666818,10.302375,1.666818,4997675,0.011653,58239.0
1,0.000000,0.000000,0.001683,0.001683,0.001933,0.002193,0.002317,0.002358,0.002706,0.00286,...,24170.0,25871.0,39614.0,58239.0,1.364222,15.453562,1.364222,4997675,0.011653,58239.0
2,0.000000,0.000000,0.001683,0.001683,0.001933,0.002193,0.002317,0.002358,0.002706,0.00286,...,24170.0,25871.0,39614.0,58239.0,1.525489,20.604749,1.525489,4997675,0.011653,58239.0
3,0.000000,0.000000,0.001683,0.001683,0.001933,0.002193,0.002317,0.002358,0.002706,0.00286,...,24170.0,25871.0,39614.0,58239.0,1.768073,22.321812,1.768073,4997675,0.011653,58239.0
4,0.000000,0.000000,0.001683,0.001683,0.001933,0.002193,0.002317,0.002358,0.002706,0.00286,...,24170.0,25871.0,39614.0,58239.0,2.286651,25.755937,2.286651,4997675,0.011653,58239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304412,0.010158,0.010594,0.012770,0.013061,0.187911,0.028272,0.000000,0.000000,0.003942,0.01774,...,3431.0,3044.0,4892.0,6891.0,1.387345,261.210274,1.387345,576641,0.011950,6891.0
304413,0.010158,0.010594,0.012770,0.013061,0.187911,0.028272,0.000000,0.000000,0.003942,0.01774,...,3431.0,3044.0,4892.0,6891.0,0.867091,261.210274,0.867091,576641,0.011950,6891.0
304414,0.010158,0.010594,0.012770,0.013061,0.187911,0.028272,0.000000,0.000000,0.003942,0.01774,...,3431.0,3044.0,4892.0,6891.0,0.693672,261.210274,0.693672,576641,0.011950,6891.0
304415,0.010158,0.010594,0.012770,0.013061,0.187911,0.028272,0.000000,0.000000,0.003942,0.01774,...,3431.0,3044.0,4892.0,6891.0,0.693672,261.210274,0.693672,576641,0.011950,6891.0


In [9]:
from sklearn.ensemble import RandomForestRegressor
# train time for random split at 0.1 test_size? 2 min
clf = RandomForestRegressor(random_state=666, max_depth=6)
clf.fit(X_train, y_train)


# from sklearn import linear_model
# clf = linear_model.Lasso(alpha=0.1)
# clf.fit(X_train, y_train)
# print(clf.coef_)
# print(clf.intercept_)

import numpy as np
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, y_train)
reg.coef_
# reg.intercept_

array([-2220184.872860, 2313.639391, -2220324.139269, 2436.100616,
       -1997.129804, 131.056916, 1085.427693, 1169.798450, -641.564670,
       849.813004, 1092.349478, 456.151604, -2000.855775, 3181.899633,
       2470.420413, 1087.542016, -8237.981470, 0.431254, -645.117930,
       -2220323.134401, 2430.432709, -7.385916, 126.210900, 477.775833,
       134.925547, -5.807311, -8.655677, 131.896333, -1.406287, 11.274092,
       19.391415, 131.102972, 131.016699, -0.596703, -8234.554585,
       -8228.455287, 845.233627, 156.340025, 5.296418, -761.713386,
       81.197594, 2.420192, 129.008406, 81.982021, 1.426316, -637.914859,
       -714.840613, -1.472565, 156.406751, -2220325.642775, 479.477843,
       -1110163.060220, -1.345083, 73.467196, -91.333147, 13.110590,
       -3.638673, 1.529876, 1.495000, 3178.600174, -707.723123,
       1162.934453, -6.465379, 2436.095927, 454.006765, 160.300030,
       6102.968881, -0.915031, -2702.580038, 2.925455, 476.521576,
       1.786058, -3.1697

In [33]:
coef = abs(reg.coef_)
# print(coef, abs(coef))
print(coef)

print(sorted(coef, reverse=True))
ind = np.argpartition(coef, -10)[-10:]
top = coef[ind]

print(ind,top)

[2220184.872860 2313.639391 2220324.139269 2436.100616 1997.129804
 131.056916 1085.427693 1169.798450 641.564670 849.813004 1092.349478
 456.151604 2000.855775 3181.899633 2470.420413 1087.542016 8237.981470
 0.431254 645.117930 2220323.134401 2430.432709 7.385916 126.210900
 477.775833 134.925547 5.807311 8.655677 131.896333 1.406287 11.274092
 19.391415 131.102972 131.016699 0.596703 8234.554585 8228.455287
 845.233627 156.340025 5.296418 761.713386 81.197594 2.420192 129.008406
 81.982021 1.426316 637.914859 714.840613 1.472565 156.406751
 2220325.642775 479.477843 1110163.060220 1.345083 73.467196 91.333147
 13.110590 3.638673 1.529876 1.495000 3178.600174 707.723123 1162.934453
 6.465379 2436.095927 454.006765 160.300030 6102.968881 0.915031
 2702.580038 2.925455 476.521576 1.786058 3.169768 3.396685 3.248332
 458.093944 81.446058 89.017889 1.059451 1166.627908 0.150170 2.996112
 714.791555 1.601146 5.111546 2.939516 2.037529 316.635700 5.325917
 2.704775 1.729030 1422.562192 3.7

In [34]:
X_train.iloc[:, ind]

,POP_HISP_x12,POP_NH,Pop,POP_x2,POP_WHITENH,POP_BLACKNH,POP_OTH2PLNH,POP_ASIANNH,POP_AIANNH,POP_NHPINH
0,0.030478,0.969522,58239.0,58239.0,0.730971,0.196844,0.028915,0.011109,0.001683,0.000000
1,0.030478,0.969522,58239.0,58239.0,0.730971,0.196844,0.028915,0.011109,0.001683,0.000000
2,0.030478,0.969522,58239.0,58239.0,0.730971,0.196844,0.028915,0.011109,0.001683,0.000000
3,0.030478,0.969522,58239.0,58239.0,0.730971,0.196844,0.028915,0.011109,0.001683,0.000000
4,0.030478,0.969522,58239.0,58239.0,0.730971,0.196844,0.028915,0.011109,0.001683,0.000000
...,...,...,...,...,...,...,...,...,...,...
304412,0.018575,0.981425,6891.0,6891.0,0.881294,0.003628,0.066028,0.007546,0.012770,0.010158
304413,0.018575,0.981425,6891.0,6891.0,0.881294,0.003628,0.066028,0.007546,0.012770,0.010158
304414,0.018575,0.981425,6891.0,6891.0,0.881294,0.003628,0.066028,0.007546,0.012770,0.010158
304415,0.018575,0.981425,6891.0,6891.0,0.881294,0.003628,0.066028,0.007546,0.012770,0.010158


In [12]:
from epsampling.utils import get_performance

y_pred = reg.predict(X_test)

model_names = ['Deaths_pred', 'Deaths_naive']
model_preds = [y_pred, y_naive]

metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
metrics_dict

{'Deaths_pred': {'MAE': 4.856512536818027,
  'MSE': 161.32593542975604,
  'RMSE': 12.701414701904511,
  'r2': 0.017524461668948055,
  'relMAE': 1.9819496067879394},
 'Deaths_naive': {'MAE': 4.650093902653675,
  'MSE': 166.89361440598498,
  'RMSE': 12.91873114535576,
  'r2': -0.01638272371233107,
  'relMAE': 1.0}}